In [ ]:
# Network Engineering in the AI Era

## Module 4.2 - Introduction to Agentic AI

Welcome to this module on Agentic AI using LangGraph!
In this notebook, we'll explore what agents are, how they operate, and build a practical example of an AI agent that can help with network configuration tasks.

### What are AI Agents?

AI agents are autonomous or semi-autonomous systems that can:

- Perceive their environment through inputs
- Reason about the information they receive
- Plan a course of action
- Execute actions to achieve specific goals
- Learn from feedback and experiences

Unlike simple models that just respond to prompts, agents can maintain state, make decisions, and take actions in a sequence to accomplish complex tasks.

Key Components of AI Agents
- Memory: Ability to store and recall information from previous interactions
- Tools: Functions or APIs that the agent can use to interact with external systems
- Planning: Ability to break down complex tasks into manageable steps
- Reasoning: Capability to make decisions based on available information
- Reflection: Ability to evaluate its own performance and adjust strategies

### LangGraph: Building Stateful Multi-Agent Systems

LangGraph is a library for building stateful, multi-agent applications with LLMs.
It extends LangChain's capabilities by providing:

- A framework for creating directed graphs where nodes represent agent states
- Tools for managing transitions between states
- Methods for handling complex workflows with multiple agents

Let's start by installing the necessary libraries:

In [3]:
!pip install langchain langchain-anthropic langchain-aws langchain-community langchain-openai langgraph chromadb faiss-cpu

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached yarl-1.18.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (69 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.0 MB/s eta 0:00:00a 0:00:01
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
Using cached frozenlist-1.5.0-cp311-cp311-macosx_11_0_arm64.whl (52 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached yarl-1.18.3-cp311-cp311-macosx_11_0_arm64.whl (92 kB)
Using cached mypy_extensions-

### Setting Up Our Environment

In [ ]:
# from typing import TypedDict, Annotated, Sequence, Any
from langchain_core.messages import HumanMessage, SystemMessage

# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS

# from langchain_core.output_parsers import StrOutputParser
from langchain_aws import ChatBedrock

llm = ChatBedrock(model="us.anthropic.claude-3-7-sonnet-20250219-v1:0", temperature=0)

### Creating Our Network Policy Knowledge Base

First, let's create a simple knowledge base of network policies that our agent can reference:

In [20]:
from langchain_aws import BedrockEmbeddings
from langchain_core.documents import Document

network_policies = [
    Document(
        page_content="Firewall Policy: All internal servers must restrict SSH access to the management VLAN only.",
        metadata={"policy_id": "FW001", "category": "firewall"},
    ),
    Document(
        page_content="VLAN Policy: Guest networks must be isolated in VLAN 100 with no access to internal resources.",
        metadata={"policy_id": "VLAN001", "category": "vlan"},
    ),
    Document(
        page_content="Router Policy: BGP peering must use MD5 authentication with unique keys per peer.",
        metadata={"policy_id": "RTR001", "category": "router"},
    ),
    Document(
        page_content="Switch Policy: All access ports must have port security enabled with a maximum of 2 MAC addresses.",
        metadata={"policy_id": "SW001", "category": "switch"},
    ),
    Document(
        page_content="Wireless Policy: All wireless networks must use WPA3 encryption with enterprise authentication.",
        metadata={"policy_id": "WLAN001", "category": "wireless"},
    ),
    Document(
        page_content="Security Policy: All network devices must have default passwords changed and stored in the password manager.",
        metadata={"policy_id": "SEC001", "category": "security"},
    ),
    Document(
        page_content="Backup Policy: Configuration backups must be performed before and after any changes to network devices.",
        metadata={"policy_id": "BKP001", "category": "backup"},
    ),
    Document(
        page_content="Access Control Policy: Administrative access to network devices must use TACACS+ with role-based permissions.",
        metadata={"policy_id": "ACC001", "category": "access"},
    ),
]

# Create a vector store
embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")
vector_store = FAISS.from_documents(network_policies, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

### Defining Our Agent's State

In LangGraph, we need to define the state that our agent will maintain throughout its execution:

In [2]:
from langgraph.graph import MessagesState
from langchain_core.documents import Document


class AgentState(MessagesState):
    """Represents the state of our network configuration agent."""

    query: str  # User's original query
    relevant_policies: list[Document]  # Retrieved policies
    config_plan: str  # Generated configuration plan
    config_commands: str  # Actual configuration commands
    execution_status: str  # Status of the execution

### Building Our Agent's Components

Now, let's define the different components (nodes) of our agent:

1. Query Understanding

In [16]:
def understand_query(state: AgentState) -> AgentState:
    """Understand the user's query and reformulate if needed."""

    messages = [
        SystemMessage(
            content="You are a network configuration assistant. Understand the user's request and reformulate it if needed."
        ),
        HumanMessage(content=state["query"]),
    ]

    response = llm.invoke(messages)

    return {
        **state,
        "messages": state["messages"]
        + [
            {"role": "system", "content": "Query understanding complete."},
            {"role": "assistant", "content": response.content},
        ],
    }

### Policy Retrieval

In [4]:
def retrieve_policies(state: AgentState) -> AgentState:
    """Retrieve relevant network policies based on the query."""

    # Use the retriever to get relevant policies
    relevant_docs = retriever.invoke(state["query"])

    # Format the policies for better readability
    policy_text = "\n\n".join(
        [f"Policy {i + 1}: {doc.page_content}" for i, doc in enumerate(relevant_docs)]
    )

    return {
        **state,
        "relevant_policies": relevant_docs,
        "messages": state["messages"]
        + [
            {
                "role": "system",
                "content": f"Retrieved relevant policies:\n\n{policy_text}",
            }
        ],
    }

### Configuration Planning

In [5]:
def plan_configuration(state: AgentState) -> AgentState:
    """Create a configuration plan based on the query and policies."""

    # Extract policy content
    policy_text = "\n".join([doc.page_content for doc in state["relevant_policies"]])

    messages = [
        SystemMessage(
            content="""You are a network configuration planner. 
        Based on the user's request and relevant policies, create a detailed plan for configuration changes.
        Your plan should be specific, step-by-step, and adhere to all relevant policies."""
        ),
        HumanMessage(
            content=f"""
        User request: {state["query"]}
        
        Relevant policies:
        {policy_text}
        
        Please create a detailed configuration plan:
        """
        ),
    ]

    response = llm.invoke(messages)

    return {
        **state,
        "config_plan": response.content,
        "messages": state["messages"]
        + [
            {"role": "system", "content": "Configuration plan created."},
            {"role": "assistant", "content": response.content},
        ],
    }

4. Generate Configuration Commands

In [6]:
def generate_commands(state: AgentState) -> AgentState:
    """Generate actual configuration commands based on the plan."""

    messages = [
        SystemMessage(
            content="""You are a network configuration expert.
        Based on the configuration plan, generate the actual commands that would be executed on the network devices.
        Use proper syntax for the relevant network devices (Cisco, Juniper, etc.)."""
        ),
        HumanMessage(
            content=f"""
        Configuration plan:
        {state["config_plan"]}
        
        Please generate the actual configuration commands:
        """
        ),
    ]

    response = llm.invoke(messages)

    return {
        **state,
        "config_commands": response.content,
        "messages": state["messages"]
        + [
            {"role": "system", "content": "Configuration commands generated."},
            {"role": "assistant", "content": response.content},
        ],
    }

5. Simulate Execution

In [7]:
def simulate_execution(state: AgentState) -> AgentState:
    """Simulate the execution of the configuration commands."""

    messages = [
        SystemMessage(
            content="""You are a network device simulator.
        Simulate the execution of the provided configuration commands and report the results.
        Include any warnings, errors, or success messages that would typically appear."""
        ),
        HumanMessage(
            content=f"""
        Configuration commands to execute:
        {state["config_commands"]}
        
        Please simulate the execution and report the results:
        """
        ),
    ]

    response = llm.invoke(messages)

    return {
        **state,
        "execution_status": response.content,
        "messages": state["messages"]
        + [
            {"role": "system", "content": "Execution simulated."},
            {"role": "assistant", "content": response.content},
        ],
    }

6. Generate Final Response

In [8]:
def generate_response(state: AgentState) -> AgentState:
    """Generate a final response to the user."""

    messages = [
        SystemMessage(
            content="""You are a helpful network configuration assistant.
        Provide a comprehensive summary of the actions taken, including:
        1. The user's original request
        2. The relevant policies considered
        3. The configuration plan created
        4. The commands that were executed
        5. The results of the execution
        
        Be professional but friendly in your response."""
        ),
        HumanMessage(
            content=f"""
        User request: {state["query"]}
        Relevant policies: {[doc.page_content for doc in state["relevant_policies"]]}
        Configuration plan: {state["config_plan"]}
        Configuration commands: {state["config_commands"]}
        Execution results: {state["execution_status"]}
        """
        ),
    ]

    response = llm.invoke(messages)

    return {
        **state,
        "messages": state["messages"]
        + [{"role": "assistant", "content": response.content}],
    }

Building the Agent Graph
Now, let's connect all these components into a graph using LangGraph:

In [10]:
from langgraph.graph import StateGraph, END

# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("understand_query", understand_query)
workflow.add_node("retrieve_policies", retrieve_policies)
workflow.add_node("plan_configuration", plan_configuration)
workflow.add_node("generate_commands", generate_commands)
workflow.add_node("simulate_execution", simulate_execution)
workflow.add_node("generate_response", generate_response)

# Add edges
workflow.add_edge("understand_query", "retrieve_policies")
workflow.add_edge("retrieve_policies", "plan_configuration")
workflow.add_edge("plan_configuration", "generate_commands")
workflow.add_edge("generate_commands", "simulate_execution")
workflow.add_edge("simulate_execution", "generate_response")
workflow.add_edge("generate_response", END)

# Set the entry point
workflow.set_entry_point("understand_query")

# Compile the graph
network_agent = workflow.compile()

Using Our Network Configuration Agent
Let's create a function to interact with our agent:

In [11]:
def process_network_request(query: str) -> dict:
    """Process a network configuration request using our agent."""

    # Initialize the state
    initial_state = {
        "messages": [],
        "query": query,
        "relevant_policies": [],
        "config_plan": "",
        "config_commands": "",
        "execution_status": "",
    }

    # Run the agent
    result = network_agent.invoke(initial_state)

    return result

Example Usage
Let's test our agent with a few examples:

In [ ]:
# Example 1: Configure a new VLAN
result = process_network_request(
    "I need to set up a new guest WiFi network that's isolated from our internal network"
)

{'messages': [SystemMessage(content='Query understanding complete.', additional_kwargs={}, response_metadata={}, id='f6998f67-c6d9-459f-9926-197428dd78ff'), AIMessage(content='# Guest WiFi Network Setup\n\nI\'ll help you set up an isolated guest WiFi network. This is a common requirement to provide internet access to visitors while keeping your internal network secure.\n\n## What you\'ll need to do:\n\n1. **Access your router/access point settings**\n   - Log into your router\'s admin interface (typically via web browser)\n\n2. **Create a new SSID**\n   - Set up a separate network name for guests\n   - Configure a different password from your main network\n\n3. **Enable network isolation/guest network features**\n   - Look for settings like "AP isolation," "Guest network," or "VLAN"\n   - Ensure clients on this network cannot access your internal devices\n\n4. **Configure internet-only access**\n   - Allow access to the internet but block access to internal resources\n   - Consider ban

In [32]:
for message in result["messages"]:
    # print(message.type)
    # print(message.content)
    print(
        f"================================= {message.type.upper()} ================================="
    )
    print(message.content)
    print("=====================================================")

================================= SYSTEM =================================
Query understanding complete.
================================= AI =================================
# Guest WiFi Network Setup

I'll help you set up an isolated guest WiFi network. This is a common requirement to provide internet access to visitors while keeping your internal network secure.

## What you'll need to do:

1. **Access your router/access point settings**
   - Log into your router's admin interface (typically via web browser)

2. **Create a new SSID**
   - Set up a separate network name for guests
   - Configure a different password from your main network

3. **Enable network isolation/guest network features**
   - Look for settings like "AP isolation," "Guest network," or "VLAN"
   - Ensure clients on this network cannot access your internal devices

4. **Configure internet-only access**
   - Allow access to the internet but block access to internal resources
   - Consider bandwidth limitations if 